In [1]:
from jupyter_dash import JupyterDash
from dash import Dash, html,dcc,Input, Output
import dash_leaflet as dl
import pandas as pd
import folium
import xyzservices.providers as xyz
MAPBOX_API_KEY = open('API_KEY', 'r').read()

In [2]:
app = Dash()
app.layout = dl.Map(dl.TileLayer(), center=[56,10], zoom=6, style={'height': '50vh'})

if __name__ == '__main__':
    app.run_server()

In [3]:
points = [
    ["720 Briar Hill Ave", 43.706843167968444, -79.43343763536379, "https://i.imgur.com/qHeXMDn.jpg", "Your travelling spirit seems to have rubbed off on me", "canada"],
    ["The Big Apple", 44.06580278508552, -77.91405655898555, "https://i.imgur.com/ly25nux.jpg", "Quite a LOVEly roadside attraction", "canada"],
    ["Tea Ceremony", 35.711900, 139.792800, "https://i.imgur.com/W7Po077.jpg", "", "japan"],
    ["Tokyo Skytree", 35.709400, 139.808600, "https://i.imgur.com/g5xFE4w.jpg", "", "japan"],
    ["Box Burger", 35.256200, 139.048000, "https://i.imgur.com/jlhhitN.jpg", "", "japan"],
    ["Kiyomizu-dera", 34.994600, 135.785400, "https://i.imgur.com/Npa3uYv.jpg", "", "japan"],
    ["Masuyacho", 34.997400, 135.781100, "https://i.imgur.com/RO81Kc8.jpg", "", "japan"],
    ["Hiroshima Dreamination", 34.388200, 132.461200, "https://i.imgur.com/i4RVUMD.jpg", "", "japan"],
    ["Miyajima", 34.279200, 132.325100, "https://i.imgur.com/QnYCUoG.jpg", "", "japan"],
    ["Hahoe Folk Village", 36.538600, 128.517000, "https://i.imgur.com/3tDDosH.jpg", "", "korea_south"],
    ["Third Infiltration Tunnel, DMZ", 37.91671648025547, 126.69829308650752, "https://i.imgur.com/t4y01pG.jpg", "", "korea_south"],
    ["Bukchon Hanok Village", 37.5828025947213, 126.98361926829193, "https://i.imgur.com/X9bGRho.jpg", "", "korea_south"],
    ["The Ex", 43.635400, -79.413600, "https://i.imgur.com/OinvUP5.jpg", "", "canada"],
    ["Eataly", 43.66977878442531, -79.38867336311971, "https://i.imgur.com/NOLqlIJ.jpg", "", "canada"],
    ["SkyDome", 43.640947, -79.389594, "https://i.imgur.com/LIf3OMh.jpg", "", "canada"]
]

maybes = [
    ["Belmanor", 44.526464496121, -80.35017367446154, "https://i.imgur.com/OlHUw0d.jpg", "", "canada"],
    ["The Melting Point", 34.862678, -111.785881, "https://i.imgur.com/YaMTx4v.jpg", "", "united_states_of_america"],
    ["The Guggenheim", 40.782936, -73.959124, "https://i.imgur.com/MbSRl11.jpg", "", "united_states_of_america"],
    ["Alpine", 44.51520990123986, -80.35446874749427, "https://i.imgur.com/EY1OCO4.jpg", "", "canada"],
    ["Odawara Station", 35.255500, 139.156000, "https://i.imgur.com/yEGPE79.jpg", "", "japan"],
]

In [4]:
df = pd.DataFrame(points + maybes, columns=['location', 'latitude', 'longitude', "image", "caption", "country"])
df['hover_info'] = df.apply(lambda row: f'''<h4 style="text-align: center; font-family: Futura; font-weight: bold;">{row["location"]}</h4><img src='{row["image"]}' alt="{row['location']}"style="max-width:200px;"><p style="text-align: center; font-family: garamond; font-size: 14px;">{row["caption"]}</p>''', axis=1)
df = df.drop(columns=['location', 'image', 'caption'])

In [5]:
TORONTO_COORDINATES = (43.66977878442531, -79.38867336311971)
m = folium.Map(location = TORONTO_COORDINATES, 
               zoom_start = 12,
               min_zoom = 2,
               max_bounds=True)

tile_provider = xyz.MapBox
tile_provider['accessToken'] = MAPBOX_API_KEY
tile_provider['url'] = 'https://api.mapbox.com/styles/v1/logandaniels/cluryho1d01wt01pbdqtxdp21/tiles/{z}/{x}/{y}{r}?access_token=' + MAPBOX_API_KEY
tile_provider['attribution'] = ' '
tile_provider['html_attribution'] = ' '
tile_provider['min_zoom'] = '2'

folium.TileLayer(
    tiles = tile_provider.build_url(api_key = MAPBOX_API_KEY),
    attr = tile_provider['attribution'],
    name = tile_provider['name'],
    max_zoom = tile_provider['max_zoom'],
    detect_retina = True,
    min_zoom = tile_provider['min_zoom']
).add_to(m)

#gjson = gdf.to_crs(epsg='4326').to_json()

for marker in range(len(df)):
    url = "https://img.freeflagicons.com/thumb/round_pin_icon/{}/{}_640.png".format
    icon_image = url(df['country'].iloc[marker], df['country'].iloc[marker])

    icon = folium.CustomIcon(
        icon_image,
        icon_size = (100, 100),
        icon_anchor = (50, 90),
        popup_anchor = (0, -80),
    )

    folium.Marker(
        location = [df['latitude'].iloc[marker], df['longitude'].iloc[marker]], 
        icon = icon, 
        popup = df['hover_info'].iloc[marker]
    ).add_to(m)
m

In [6]:
m.save('dash_experiment.html')
app.layout = html.Div([
    html.Iframe(id = 'map', srcDoc = open('dash_experiment.html', 'r').read(), width = '100%', height = 600)
])